In [2]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient
from collections import defaultdict

In [3]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))
db = client.get_database(os.getenv('DB_Collection'))

In [4]:
# 특정 유저의 frequent_zone이 0인 basic-unit 찾기
user_uuid = "cf49f5c9dd0f4ee6af663bf7cad14359"  # 확인하고 싶은 유저 UUID로 변경

collection_filtered = db['basicunit_clusterlabel_frequency_up100_20-24_0402']
frequent_zone_1 = collection_filtered.find({"uuid": user_uuid, "frequent_zone": 1}, {"destination_area": 1, "cluster_label":1, "_id": 0})
frequent_zone_0 = collection_filtered.find({"uuid": user_uuid, "frequent_zone": 0}, {"destination_area": 1, "cluster_label":1, "_id": 0})

mapped_category_collection = db['user_category_frequency_20-24']
user_where_collection = db['user_category_place_20-24']

# 특정 유저의 mapped_category 및 선호도 // place name 및 mapped category 조회
target_user_categories = list(mapped_category_collection.find({"uuid": user_uuid}, {"mapped_category": 1, "frequency": 1, "_id": 0}))
target_user_where = list(user_where_collection.find({"uuid": user_uuid}, {"mapped_category": 1, "place_name": 1, "_id": 0}))


In [5]:
frequent_zone_1=list(frequent_zone_1)
frequent_zone_0=list(frequent_zone_0)

In [6]:
## 자주 찾지 않는 지역에 사는 로컬들 조회회
basic_unit_mapping_collection = db['clusterlabel_uuid_mapping_20-24']

# 각 basic-unit에 해당하는 uuids 가져오기
basic_unit_uuids_map = {}

print(f"📢 UUID {user_uuid}의 frequent_zone이 0인 basic-unit 목록:")
for data in frequent_zone_0:
    print(data["destination_area"], data['cluster_label'])


# 추천 성능 검증을 위한 1 로 실험
print(f"📢 UUID {user_uuid}의 frequent_zone이 1인 basic-unit 목록:")
for data in frequent_zone_1:
    print(data["destination_area"], data['cluster_label'])
    mapping_entry = basic_unit_mapping_collection.find_one({"basic_unit": data["destination_area"], "cluster_label":data['cluster_label']}, {"uuids": 1, "_id": 0})
    uuids = mapping_entry["uuids"] if mapping_entry else []
    basic_unit_uuids_map[(data["destination_area"], data["cluster_label"])] = uuids
print("-" * 50)

### 결과값 : 각 basic-unit에 속하는 UUID 목록 및 개수


📢 UUID cf49f5c9dd0f4ee6af663bf7cad14359의 frequent_zone이 0인 basic-unit 목록:
서울 송파구 3
서울 은평구 3
서울 성북구 3
인천 남동구 2
이천시 2
횡성군 4
양평군 1
시흥시 1
서울 중구 1
고양시 일산서구 1
서울 중랑구 2
안산시 상록구 2
화성시 2
부천시 1
평택시 1
고양시 덕양구 1
성남시 분당구 1
서울 광진구 3
서울 용산구 1
서울 금천구 2
수원시 팔달구 2
수원시 팔달구 4
서울 광진구 1
가평군 1
용인시 처인구 2
서울 관악구 4
서울 강남구 2
용인시 기흥구 1
공주시 1
서울 강서구 3
광명시 1
서울 종로구 1
용인시 처인구 1
안산시 상록구 1
파주시 1
시흥시 4
성남시 분당구 2
안양시 만안구 3
성남시 중원구 2
과천시 3
광주시 1
안양시 동안구 1
성남시 수정구 3
성남시 수정구 1
광주시 4
고양시 일산동구 2
안성시 2
원주시 2
광주시 2
서울 관악구 2
평창군 1
안산시 단원구 2
서울 양천구 2
안양시 동안구 2
인천 옹진군 1
서울 구로구 2
횡성군 5
인천 미추홀구 4
인천 강화군 2
당진시 1
홍천군 1
속초시 2
서울 서초구 3
익산시 1
포천시 3
서울 영등포구 2
파주시 4
안양시 만안구 2
가평군 4
가평군 2
남양주시 3
성남시 수정구 4
수원시 영통구 4
서울 도봉구 2
군포시 2
서울 성동구 1
속초시 3
속초시 5
서울 구로구 3
고양시 일산동구 3
서울 강동구 1
서울 동작구 1
📢 UUID cf49f5c9dd0f4ee6af663bf7cad14359의 frequent_zone이 1인 basic-unit 목록:
서울 금천구 1
--------------------------------------------------


In [7]:
basic_unit_uuids_map

{('서울 금천구', 1): ['0037b972742344bb8bba86dd835bda73',
  '02ae2b0222d041a1b0dc467fddff9762',
  '03ae55f496df408ebf10fffe2da5aa0b',
  '04bb2a7f184c4b1e8bd4c20c4209eee4',
  '052b3e03e6a74007a5450f110f7d849a',
  '05311fc081444960858ee88cfdc5b4d0',
  '078e281d08d646d9a73516fdf1826b6b',
  '08c0a343aafd402e81b537f002452753',
  '08cd2b3ed8644847a21fb67babfe43c7',
  '0afd94a1a766404f89bdacc10f42207f',
  '0c2ff73b62f54e3a9f148a23b35e6f76',
  '0dd6bf5b0743447b9d3ad6001869d650',
  '0f0ba0a032a745b4a3f75b5036b589ca',
  '0f76c242cf784a38ad628b85dfd18b6c',
  '1088ba5c826248d48177c2fd98e1df26',
  '138542d8d29f423b9f24bda60ceb7af2',
  '13f01814fd174cf4af52e440f41559d6',
  '15abce93012f4b069523123053a19c2f',
  '166419300d47493da5c37f206b273f74',
  '17ba0fe0cc2d4f1cae328fa5baead1c1',
  '189bea0297a24b6a973c6e0d0195be75',
  '18be9796f0804f6b9e9044c7f89dd590',
  '1a52abe3e14e452197d4fa14b697602b',
  '1b12ae6947d641988ecf5331461c580d',
  '1b91c381c7724ea8befab5fee5044009',
  '1bdeeb3062434e07b7a30eb940dd6516

In [11]:
target_user_categories

[{'mapped_category': '양식 > 버거', 'frequency': 6},
 {'mapped_category': '한식 > 구이 & 육류 요리', 'frequency': 1},
 {'mapped_category': '한식 > 국수', 'frequency': 1},
 {'mapped_category': '한식 > 전골 & 찌개', 'frequency': 1},
 {'mapped_category': '간식', 'frequency': 1}]

In [34]:
target_categories_list[0]

('양식 > 버거', 6)

In [43]:
## 자주 찾지 않는 지역 ["basic_unit"] 에 사는 로컬들["user_uuid"]과 그들의 카테고리 선호도[ (entry["mapped_category"], entry["frequency"]) ]

# 특정 유저의 mapped_category를 set으로 변환하여 빠른 비교 가능
target_categories_set = {(entry["mapped_category"], entry["frequency"]) for entry in target_user_categories}
# target_categories_list=list(target_categories_set)
# 각 UUID에 대한 mapped_category 및 frequency 조회 및 필터링
uuid_category_map = {}
matching_users = []
basic_unit_uuid_count = {}

for basic_unit, uuids in basic_unit_uuids_map.items():
    count = 0
    uuid_category_map[basic_unit] = []
    for uuid in uuids:
        category_entries = mapped_category_collection.find({"uuid": uuid}, {"mapped_category": 1, "frequency": 1, "uuid": 1, "_id": 0})
        for entry in category_entries:
            entry["user_uuid"] = uuid  # 유저 정보 포함
            entry["basic_unit"] = basic_unit  # basic-unit 정보 포함
            uuid_category_map[basic_unit].append(entry)
            # for cate in target_categories_list:
                # if entry["mapped_category"] in cate[0]:
            if (entry["mapped_category"], entry["frequency"]) in target_categories_set:
                ### 좀 이상한디?
                ### 방문 지역민 중 카테고리의 빈도수까지 같아야됨?
                matching_users.append(entry)
                basic_unit_uuid_count[entry["basic_unit"]] = basic_unit_uuid_count.get(entry["basic_unit"], 0) + 1

### 결과값 : 📢 각 basic-unit에 속하는 UUID의 카테고리 및 빈도 정보: 📢 특정 유저와 동일한 mapped_category 및 frequency를 가진 UUID 목록:


In [44]:
matching_users

[{'uuid': '05311fc081444960858ee88cfdc5b4d0',
  'mapped_category': '간식',
  'frequency': 1,
  'user_uuid': '05311fc081444960858ee88cfdc5b4d0',
  'basic_unit': ('서울 금천구', 1)},
 {'uuid': '078e281d08d646d9a73516fdf1826b6b',
  'mapped_category': '한식 > 국수',
  'frequency': 1,
  'user_uuid': '078e281d08d646d9a73516fdf1826b6b',
  'basic_unit': ('서울 금천구', 1)},
 {'uuid': '078e281d08d646d9a73516fdf1826b6b',
  'mapped_category': '간식',
  'frequency': 1,
  'user_uuid': '078e281d08d646d9a73516fdf1826b6b',
  'basic_unit': ('서울 금천구', 1)},
 {'uuid': '0dd6bf5b0743447b9d3ad6001869d650',
  'mapped_category': '한식 > 구이 & 육류 요리',
  'frequency': 1,
  'user_uuid': '0dd6bf5b0743447b9d3ad6001869d650',
  'basic_unit': ('서울 금천구', 1)},
 {'uuid': '0dd6bf5b0743447b9d3ad6001869d650',
  'mapped_category': '간식',
  'frequency': 1,
  'user_uuid': '0dd6bf5b0743447b9d3ad6001869d650',
  'basic_unit': ('서울 금천구', 1)},
 {'uuid': '166419300d47493da5c37f206b273f74',
  'mapped_category': '간식',
  'frequency': 1,
  'user_uuid': '16641

In [45]:
len(matching_users)

47

In [41]:
len(matching_users)

0

In [37]:
##추천할 장소들 평점 순으로 나열하여 리스트 업
udrt_kakao_cdh_place_collection = db['user_category_place_20-24']
rating_collection = db['rating_details_2020-2024']

# UUID, mapped_category, basic_unit를 기반으로 place_name 찾기
uuid_mapped_category_pairs = [(entry['user_uuid'], entry['mapped_category'], entry['basic_unit']) for entry in matching_users]
print(uuid_mapped_category_pairs)
# (Basic Unit, Mapped Category) 별로 그룹화하여 place_name 리스트 생성
place_name_grouped = defaultdict(list)
for uuid, mapped_category, basic_unit in uuid_mapped_category_pairs:
    place_entries = udrt_kakao_cdh_place_collection.find({"uuid": uuid, "mapped_category": mapped_category}, {"place_name": 1, "_id": 0})
    for place in place_entries:
        place_name_grouped[(basic_unit, mapped_category)].append(place["place_name"])

# 특정 유저에게 제공할 데이터 생성 및 정렬
user_recommendations = {}
for key, place_names in place_name_grouped.items():
    rated_places = []
    for place_name in set(place_names):  # 중복 제거
        rating_entry = rating_collection.find_one({"place_name": place_name}, {"rate": 1, "_id": 0})
        rating = float(rating_entry["rate"]) if rating_entry and "rate" in rating_entry else 0.0  # 문자열을 float으로 변환
        rated_places.append((place_name, rating))
    
    # 평점이 높은 순으로 정렬
    sorted_places = [(place[0], place[1]) for place in sorted(rated_places, key=lambda x: x[1], reverse=True)]
    user_recommendations[key] = sorted_places

[('05311fc081444960858ee88cfdc5b4d0', '한식 > 국수', ('서울 금천구', 1)), ('05311fc081444960858ee88cfdc5b4d0', '한식 > 구이 & 육류 요리', ('서울 금천구', 1)), ('05311fc081444960858ee88cfdc5b4d0', '간식', ('서울 금천구', 1)), ('05311fc081444960858ee88cfdc5b4d0', '한식', ('서울 금천구', 1)), ('05311fc081444960858ee88cfdc5b4d0', '한식', ('서울 금천구', 1)), ('05311fc081444960858ee88cfdc5b4d0', '한식', ('서울 금천구', 1)), ('078e281d08d646d9a73516fdf1826b6b', '한식 > 구이 & 육류 요리', ('서울 금천구', 1)), ('078e281d08d646d9a73516fdf1826b6b', '한식 > 국수', ('서울 금천구', 1)), ('078e281d08d646d9a73516fdf1826b6b', '간식', ('서울 금천구', 1)), ('078e281d08d646d9a73516fdf1826b6b', '한식', ('서울 금천구', 1)), ('078e281d08d646d9a73516fdf1826b6b', '한식', ('서울 금천구', 1)), ('078e281d08d646d9a73516fdf1826b6b', '한식', ('서울 금천구', 1)), ('0dd6bf5b0743447b9d3ad6001869d650', '한식', ('서울 금천구', 1)), ('0dd6bf5b0743447b9d3ad6001869d650', '한식', ('서울 금천구', 1)), ('0dd6bf5b0743447b9d3ad6001869d650', '한식', ('서울 금천구', 1)), ('0dd6bf5b0743447b9d3ad6001869d650', '한식 > 구이 & 육류 요리', ('서울 금천구', 1)), ('0dd6

In [38]:
# 결과 출력
print("📢 각 basic-unit에 속하는 UUID 목록 및 개수:")
for basic_unit, uuids in basic_unit_uuids_map.items():
    print(f"Basic Unit: {basic_unit[0]}, cluster label: {basic_unit[1]} -> UUID Count: {len(uuids)}, UUIDs: {uuids}")
# print("-" * 50)

📢 각 basic-unit에 속하는 UUID 목록 및 개수:
Basic Unit: 서울 금천구, cluster label: 1 -> UUID Count: 204, UUIDs: ['0037b972742344bb8bba86dd835bda73', '02ae2b0222d041a1b0dc467fddff9762', '03ae55f496df408ebf10fffe2da5aa0b', '04bb2a7f184c4b1e8bd4c20c4209eee4', '052b3e03e6a74007a5450f110f7d849a', '05311fc081444960858ee88cfdc5b4d0', '078e281d08d646d9a73516fdf1826b6b', '08c0a343aafd402e81b537f002452753', '08cd2b3ed8644847a21fb67babfe43c7', '0afd94a1a766404f89bdacc10f42207f', '0c2ff73b62f54e3a9f148a23b35e6f76', '0dd6bf5b0743447b9d3ad6001869d650', '0f0ba0a032a745b4a3f75b5036b589ca', '0f76c242cf784a38ad628b85dfd18b6c', '1088ba5c826248d48177c2fd98e1df26', '138542d8d29f423b9f24bda60ceb7af2', '13f01814fd174cf4af52e440f41559d6', '15abce93012f4b069523123053a19c2f', '166419300d47493da5c37f206b273f74', '17ba0fe0cc2d4f1cae328fa5baead1c1', '189bea0297a24b6a973c6e0d0195be75', '18be9796f0804f6b9e9044c7f89dd590', '1a52abe3e14e452197d4fa14b697602b', '1b12ae6947d641988ecf5331461c580d', '1b91c381c7724ea8befab5fee5044009', '

In [39]:
# 결과 출력
print(f"📢 UUID {user_uuid}의 mapped_category 및 frequency:")
for entry in target_user_categories:
    print(f"Mapped Category: {entry['mapped_category']}, Frequency: {entry['frequency']}")

print(f"📢 UUID {user_uuid}의 mapped_category 및 place_name:")
for entry in target_user_where:
    print(f"User Where : {entry['mapped_category']}, Place_name: {entry['place_name']}")
# print("-" * 50)

📢 UUID cf49f5c9dd0f4ee6af663bf7cad14359의 mapped_category 및 frequency:
Mapped Category: 양식 > 버거, Frequency: 6
Mapped Category: 한식 > 구이 & 육류 요리, Frequency: 1
Mapped Category: 한식 > 국수, Frequency: 1
Mapped Category: 한식 > 전골 & 찌개, Frequency: 1
Mapped Category: 간식, Frequency: 1
📢 UUID cf49f5c9dd0f4ee6af663bf7cad14359의 mapped_category 및 place_name:
User Where : 한식 > 구이 & 육류 요리, Place_name: 황홀한막창 관설점
User Where : 양식 > 버거, Place_name: 송탄진버거
User Where : 간식, Place_name: 몽블랑제 홈플러스화성향남점
User Where : 양식 > 버거, Place_name: 송탄진버거
User Where : 양식 > 버거, Place_name: 송탄진버거
User Where : 한식 > 국수, Place_name: 사랑방칼국수
User Where : 양식 > 버거, Place_name: 송탄진버거
User Where : 한식 > 전골 & 찌개, Place_name: 명품의정부부대찌개
User Where : 양식 > 버거, Place_name: 송탄진버거
User Where : 양식 > 버거, Place_name: 송탄진버거


In [14]:
print("📢 각 basic-unit에 속하는 UUID의 카테고리 및 빈도 정보:")
for basic_unit, categories in uuid_category_map.items():
    print(f"Basic Unit: {basic_unit[0]}, cluster label: {basic_unit[1]} (Total UUIDs: {len(basic_unit_uuids_map[basic_unit])})") # 갯수
    for entry in categories:
        print(f"  User UUID: {entry['user_uuid']}, Basic Unit: {entry['basic_unit'][0]} , cluster label: {entry['basic_unit'][1]}, Mapped Category: {entry['mapped_category']}, Frequency: {entry['frequency']}")
    # print("-" * 50)

📢 각 basic-unit에 속하는 UUID의 카테고리 및 빈도 정보:
Basic Unit: 서울 금천구, cluster label: 1 (Total UUIDs: 204)
  User UUID: 05311fc081444960858ee88cfdc5b4d0, Basic Unit: 서울 금천구 , cluster label: 1, Mapped Category: 한식 > 해산물, Frequency: 31
  User UUID: 05311fc081444960858ee88cfdc5b4d0, Basic Unit: 서울 금천구 , cluster label: 1, Mapped Category: 한식 > 탕류, Frequency: 7
  User UUID: 05311fc081444960858ee88cfdc5b4d0, Basic Unit: 서울 금천구 , cluster label: 1, Mapped Category: 한식 > 국수, Frequency: 5
  User UUID: 05311fc081444960858ee88cfdc5b4d0, Basic Unit: 서울 금천구 , cluster label: 1, Mapped Category: 중식 > 중국요리, Frequency: 4
  User UUID: 05311fc081444960858ee88cfdc5b4d0, Basic Unit: 서울 금천구 , cluster label: 1, Mapped Category: 치킨, Frequency: 3
  User UUID: 05311fc081444960858ee88cfdc5b4d0, Basic Unit: 서울 금천구 , cluster label: 1, Mapped Category: 한식 > 구이 & 육류 요리, Frequency: 2
  User UUID: 05311fc081444960858ee88cfdc5b4d0, Basic Unit: 서울 금천구 , cluster label: 1, Mapped Category: 간식, Frequency: 1
  User UUID: 05311fc0814449

In [14]:
udrt_kakao_cdh_collection = db['udrt_kakao_cdh_2024']

# 해당 UUID가 방문한 place_name 및 road_address_name 목록 가져오기
user_places = udrt_kakao_cdh_collection.find({"uuid": user_uuid}, {"place_name": 1, "road_address_name": 1, "_id": 0})
user_places = list(user_places)
# (Place Name, Road Address) 매핑 생성
user_place_map = {(place["place_name"], place["road_address_name"]) for place in user_places}
user_place_map=dict(user_place_map)
# 결과 출력
print(f"📢 {user_uuid}에게 제공될 (Basic Unit, Mapped Category)별 Place Name 목록 (평점 순 정렬):")
for (basic_unit, mapped_category), place_list in user_recommendations.items():
    print("-" * 50)
    print(f"Basic Unit: {basic_unit}, Mapped Category: {mapped_category} ({len(place_list)})")
    for place_name, rating in place_list:
        print(f"  Place: {place_name}, Rating: {rating}")

📢 cf49f5c9dd0f4ee6af663bf7cad14359에게 제공될 (Basic Unit, Mapped Category)별 Place Name 목록 (평점 순 정렬):
--------------------------------------------------
Basic Unit: ('서울 금천구', 1), Mapped Category: 간식 (19)
  Place: 핑크벌스데이, Rating: 5.0
  Place: 속초불탄감자빵 1호점, Rating: 5.0
  Place: 시장떡집, Rating: 4.8
  Place: 베이커리 무이, Rating: 4.5
  Place: 엔젤하우스, Rating: 4.4
  Place: 욥, Rating: 4.4
  Place: 뱅뱅탕후루, Rating: 4.0
  Place: 설리케이크, Rating: 4.0
  Place: 빵야베이커리, Rating: 4.0
  Place: 명장시대, Rating: 3.9
  Place: 보네스뻬 삼산점, Rating: 3.8
  Place: 몽블랑제 홈플러스익스프레스 서울오류점, Rating: 3.8
  Place: 메고지고 상록수점, Rating: 3.8
  Place: 수랑아뜰리에, Rating: 3.6
  Place: 몽블랑제 홈플러스화성향남점, Rating: 3.5
  Place: 소망분식, Rating: 3.4
  Place: 파리바게뜨 시흥뉴타운점, Rating: 3.2
  Place: 빵명장, Rating: 2.7
  Place: 라임푸드, Rating: 2.5
--------------------------------------------------
Basic Unit: ('서울 금천구', 1), Mapped Category: 한식 > 국수 (10)
  Place: 강남면옥 광명점, Rating: 4.7
  Place: 봉평 옹심이 메밀칼국수, Rating: 4.6
  Place: 명지해물찜칼국수, Rating: 4.6
  Place: 연변냉면 대림2호점, Rat

In [15]:
user_place_map

{'송탄진버거': '경기 화성시 향남읍 발안공단로 86',
 '명품의정부부대찌개': '경기 화성시 향남읍 발안공단로 101',
 '사랑방칼국수': '서울 중구 동호로11길 30'}

In [16]:
user_recommendations

{(('서울 금천구', 1), '간식'): [('핑크벌스데이', 5.0),
  ('속초불탄감자빵 1호점', 5.0),
  ('시장떡집', 4.8),
  ('베이커리 무이', 4.5),
  ('엔젤하우스', 4.4),
  ('욥', 4.4),
  ('뱅뱅탕후루', 4.0),
  ('설리케이크', 4.0),
  ('빵야베이커리', 4.0),
  ('명장시대', 3.9),
  ('보네스뻬 삼산점', 3.8),
  ('몽블랑제 홈플러스익스프레스 서울오류점', 3.8),
  ('메고지고 상록수점', 3.8),
  ('수랑아뜰리에', 3.6),
  ('몽블랑제 홈플러스화성향남점', 3.5),
  ('소망분식', 3.4),
  ('파리바게뜨 시흥뉴타운점', 3.2),
  ('빵명장', 2.7),
  ('라임푸드', 2.5)],
 (('서울 금천구', 1), '한식 > 국수'): [('강남면옥 광명점', 4.7),
  ('봉평 옹심이 메밀칼국수', 4.6),
  ('명지해물찜칼국수', 4.6),
  ('연변냉면 대림2호점', 4.0),
  ('경주아화전통국수 동천점', 3.8),
  ('촌집막국수', 3.5),
  ('소표국수 낙성대점', 3.5),
  ('사랑방칼국수', 3.1),
  ('국수나무 봉천현대시장점', 2.7),
  ('팔당냉면 광명뉴코아점', 2.0)],
 (('서울 금천구', 1), '한식 > 구이 & 육류 요리'): [('삼삼한구이', 5.0),
  ('이층고깃집 수원영통점', 4.8),
  ('종로곱창', 4.5),
  ('황홀한막창 관설점', 4.4),
  ('두근반 상동점', 4.2),
  ('소고기굼터', 3.1),
  ('정훈닭치킨', 3.0)],
 (('서울 금천구', 1), '한식 > 전골 & 찌개'): [('명품의정부부대찌개', 5.0),
  ('양푼이김치찌개', 5.0),
  ('놀부부대찌개 홈플러스금천점', 4.6),
  ('땅스부대찌개 태평시장점', 4.6),
  ('마이스터심슨부대찌개 중대물빛공원점', 4.4),
  ('탕가네 서울대

In [17]:
user_place_map

{'송탄진버거': '경기 화성시 향남읍 발안공단로 86',
 '명품의정부부대찌개': '경기 화성시 향남읍 발안공단로 101',
 '사랑방칼국수': '서울 중구 동호로11길 30'}

In [21]:
# (Basic Unit, Mapped Category)별로 Place Name 및 Road Address 매칭하여 출력
true_positive = 0
false_positive = 0
false_negative = 0

print(f"📢 {user_uuid}의 (Basic Unit, Mapped Category)별 Place Name 및 Road Address 일치하는 목록:")
for (basic_unit, mapped_category), place_list in user_recommendations.items():
    matched_places = [(place_name, road_address) for place_name, road_address in user_place_map.items() 
                      if place_name in [p[0] for p in place_list] and basic_unit[0] in road_address]
    contains_count = len(matched_places)  # 포함됨 개수 카운트
    
    if contains_count > 0:
        print(f"Basic Unit: {basic_unit}, Mapped Category: {mapped_category}")
        for place_name, road_address in matched_places:
            print(f"  Place: {place_name}, Road Address: {road_address} ✅ 포함됨")
        print(f"📢 포함됨 개수: {contains_count}")
        print(f"추천된 장소 리스트의 갯수 : {len(place_list)} 실제로 방문한 장소 리스트의 갯수 : {contains_count}")
        print("-" * 50)
        
        true_positive += contains_count
        false_positive += len(place_list) - contains_count
        false_negative += len(user_place_map) - contains_count

# Precision, Recall, F1-Score 계산
precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"📢 Precision: {precision:.4f}")
print(f"📢 Recall: {recall:.4f}")
print(f"📢 F1-Score: {f1_score:.4f}")


📢 cf49f5c9dd0f4ee6af663bf7cad14359의 (Basic Unit, Mapped Category)별 Place Name 및 Road Address 일치하는 목록:
📢 Precision: 0.0000
📢 Recall: 0.0000
📢 F1-Score: 0.0000
